In [34]:
from transformers import Wav2Vec2Processor, Wav2Vec2ForCTC
from datasets import load_dataset
import torch

# load model and processor
processor = Wav2Vec2Processor.from_pretrained(
    "facebook/wav2vec2-base-960h")
model = Wav2Vec2ForCTC.from_pretrained(
    "facebook/wav2vec2-base-960h")
# feature_extractor = Wav2Vec2ForCTC.from_pretrained(
#     "facebook/wav2vec2-base-960h")

# print(feature_extractor.sampling_rate)

# model.save_pretrained("/root/ws/ml/local-inference/model")
# processor.save_pretrained("/root/ws/ml/local-inference/model")

# # load dummy dataset and read soundfiles
# ds = load_dataset("patrickvonplaten/librispeech_asr_dummy",
#                   "clean", split="validation")


Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Wav2Vec2Config {
  "_name_or_path": "facebook/wav2vec2-base-960h",
  "activation_dropout": 0.1,
  "adapter_kernel_size": 3,
  "adapter_stride": 2,
  "add_adapter": false,
  "apply_spec_augment": true,
  "architectures": [
    "Wav2Vec2ForCTC"
  ],
  "attention_dropout": 0.1,
  "bos_token_id": 1,
  "classifier_proj_size": 256,
  "codevector_dim": 256,
  "contrastive_logits_temperature": 0.1,
  "conv_bias": false,
  "conv_dim": [
    512,
    512,
    512,
    512,
    512,
    512,
    512
  ],
  "conv_kernel": [
    10,
    3,
    3,
    3,
    3,
    2,
    2
  ],
  "conv_stride": [
    5,
    2,
    2,
    2,
    2,
    2,
    2
  ],
  "ctc_loss_reduction": "sum",
  "ctc_zero_infinity": false,
  "diversity_loss_weight": 0.1,
  "do_stable_layer_norm": false,
  "eos_token_id": 2,
  "feat_extract_activation": "gelu",
  "feat_extract_dropout": 0.0,
  "feat_extract_norm": "group",
  "feat_proj_dropout": 0.1,
  "feat_quantizer_dropout": 0.0,
  "final_dropout": 0.1,
  "gradient_checkpointin

In [22]:
from datasets import Dataset, interleave_datasets
from datasets import Audio

# ds = Dataset.from_dict({"audio": ["/root/ws/ml/videos/rachel-last-night-working-here/scene-audio.mp3"]}).cast_column("audio", Audio(sampling_rate=16_000))
ds = Dataset.from_dict({"audio": ["/tmp/audiod00c40c5e11de3ecc94e6d0cf47421154a61042d-0-100.wav"]}).cast_column("audio", Audio(sampling_rate=16_000))
ds[0]["audio"]

# tokenize
input_values = processor(
    ds[0]["audio"]["array"], return_tensors="pt", padding="longest").input_values


It is strongly recommended to pass the ``sampling_rate`` argument to this function. Failing to do so can result in silent errors that might be hard to debug.


In [23]:
import numpy as np
import IPython.display as ipd

print(ds[0]["audio"])
print(ds[0]["audio"]["array"])
print(np.asarray(ds[0]["audio"]["array"]))

ipd.Audio(data=np.asarray(ds[0]["audio"]["array"]), autoplay=True, rate=16000)

{'path': '/tmp/audiod00c40c5e11de3ecc94e6d0cf47421154a61042d-0-100.wav', 'array': array([ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00, ...,
       -3.2329528e-05,  1.9558120e-08, -5.6106592e-07], dtype=float32), 'sampling_rate': 16000}
[ 0.0000000e+00  0.0000000e+00  0.0000000e+00 ... -3.2329528e-05
  1.9558120e-08 -5.6106592e-07]
[ 0.0000000e+00  0.0000000e+00  0.0000000e+00 ... -3.2329528e-05
  1.9558120e-08 -5.6106592e-07]


In [24]:
print(model.config.inputs_to_logits_ratio)

320


In [25]:
# retrieve logits
logits = model(input_values).logits

# take argmax and decode
predicted_ids = torch.argmax(logits, dim=-1)
transcription = processor.batch_decode(predicted_ids, output_word_offsets = True)

transcription


Wav2Vec2CTCTokenizerOutput(text=['AS AT THIS MOMENT I WILL NEVER HAVE TO MAKE COFFEE AGAIN'], char_offsets=None, word_offsets=[[{'word': 'AS', 'start_offset': 26, 'end_offset': 30}, {'word': 'AT', 'start_offset': 37, 'end_offset': 39}, {'word': 'THIS', 'start_offset': 47, 'end_offset': 56}, {'word': 'MOMENT', 'start_offset': 65, 'end_offset': 83}, {'word': 'I', 'start_offset': 93, 'end_offset': 94}, {'word': 'WILL', 'start_offset': 96, 'end_offset': 106}, {'word': 'NEVER', 'start_offset': 114, 'end_offset': 128}, {'word': 'HAVE', 'start_offset': 136, 'end_offset': 146}, {'word': 'TO', 'start_offset': 149, 'end_offset': 153}, {'word': 'MAKE', 'start_offset': 156, 'end_offset': 164}, {'word': 'COFFEE', 'start_offset': 173, 'end_offset': 193}, {'word': 'AGAIN', 'start_offset': 219, 'end_offset': 237}]])

In [32]:
# pred_ids = torch.argmax(logits, axis=-1)
# outputs = processor.decode(pred_ids, output_word_offsets=True)
time_offset = model.config.inputs_to_logits_ratio / 16000
print(model.config.inputs_to_logits_ratio)
print(time_offset)
print(transcription.word_offsets)

word_offsets = [
    {
        "word": d["word"],
        "start_time": round(d["start_offset"] * time_offset, 2),
        "end_time": round(d["end_offset"] * time_offset, 2),
    }
    for d in transcription.word_offsets[0]
]

word_offsets

320
0.02
[[{'word': 'AS', 'start_offset': 26, 'end_offset': 30}, {'word': 'AT', 'start_offset': 37, 'end_offset': 39}, {'word': 'THIS', 'start_offset': 47, 'end_offset': 56}, {'word': 'MOMENT', 'start_offset': 65, 'end_offset': 83}, {'word': 'I', 'start_offset': 93, 'end_offset': 94}, {'word': 'WILL', 'start_offset': 96, 'end_offset': 106}, {'word': 'NEVER', 'start_offset': 114, 'end_offset': 128}, {'word': 'HAVE', 'start_offset': 136, 'end_offset': 146}, {'word': 'TO', 'start_offset': 149, 'end_offset': 153}, {'word': 'MAKE', 'start_offset': 156, 'end_offset': 164}, {'word': 'COFFEE', 'start_offset': 173, 'end_offset': 193}, {'word': 'AGAIN', 'start_offset': 219, 'end_offset': 237}]]


[{'word': 'AS', 'start_time': 0.52, 'end_time': 0.6},
 {'word': 'AT', 'start_time': 0.74, 'end_time': 0.78},
 {'word': 'THIS', 'start_time': 0.94, 'end_time': 1.12},
 {'word': 'MOMENT', 'start_time': 1.3, 'end_time': 1.66},
 {'word': 'I', 'start_time': 1.86, 'end_time': 1.88},
 {'word': 'WILL', 'start_time': 1.92, 'end_time': 2.12},
 {'word': 'NEVER', 'start_time': 2.28, 'end_time': 2.56},
 {'word': 'HAVE', 'start_time': 2.72, 'end_time': 2.92},
 {'word': 'TO', 'start_time': 2.98, 'end_time': 3.06},
 {'word': 'MAKE', 'start_time': 3.12, 'end_time': 3.28},
 {'word': 'COFFEE', 'start_time': 3.46, 'end_time': 3.86},
 {'word': 'AGAIN', 'start_time': 4.38, 'end_time': 4.74}]